In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [3]:
import numpy as np
import scanpy as sc
import cellrank as cr
import scvelo as scv
import jax
import jax.numpy as jnp
from tueplots import bundles
from tueplots import axes
import matplotlib.pyplot as plt

from neural_ot.utils import get_neural_dual_from_ckpt

ModuleNotFoundError: No module named 'ott.core'

In [ ]:
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo', transparent=True, fontsize=20, color_map='viridis')  # for beautified visualization
scv.settings.set_figure_params('scvelo', dpi_save=400, dpi=200, transparent=True, fontsize=20, color_map='viridis')

cr.logging.print_versions()

plt.rcParams.update(**bundles.neurips2022(usetex=False, nrows=2), **axes.lines(base_width=0.5), **{"figure.dpi": 250})

In [ ]:
# params
input_dim = 50
batch_size = 1024
first_quadratic_term = False
first_quadratic = True
ckpt_dir = f"ckpts/pancreas1415_pmm+gr"

In [ ]:
# create and load neural dual
neural_dual = get_neural_dual_from_ckpt(
    ckpt_dir=ckpt_dir,
    first_quadratic_term=first_quadratic_term,
    first_quadratic=first_quadratic,
    input_dim=input_dim)

In [ ]:
adata = sc.read("../neural_ot/data/pancreas/1415_growth_rate.h5ad")

In [ ]:
# Prepare for Cellrank
adata.obs["clusters"] = adata.obs["celltype"]
adata = new_adata[:, :50].copy()
adata.layers["velocity"] = (neural_dual.transport(adata.obsm["X_pca"]) - adata.obsm["X_pca"])
adata.layers["velocity"] = np.asarray(adata.layers["velocity"].copy())
adata.layers["pca"] = adata.obsm["X_pca"].copy()

In [ ]:
vk = cr.kernels.VelocityKernel(adata, xkey="pca")
vk.compute_transition_matrix()

In [ ]:
g = cr.estimators.GPCCA(vk)

In [ ]:
g.compute_schur(n_components=20)
g.plot_spectrum(real_only=True)

In [ ]:
g.compute_macrostates(n_states=7, cluster_key="celltype")

In [ ]:
g.compute_terminal_states(method="top_n", n_states=6)
scv.set_figure_params('scvelo', transparent=True, fontsize=20, color_map='viridis')
g.plot_terminal_states(discrete=True, basis="umap", legend_loc="right", legend_fontweight='normal', legend_fontsize='12', dpi=250)

In [ ]:
g.set_terminal_states_from_macrostates(
    [
        "Acinar", "Ductal", "Alpha", "Alpha", "Beta", "Delta", "Epsilon"
    ]
)
g.compute_absorption_probabilities(solver="gmres", use_petsc=True, tol=1e-12, preconditioner='ilu', time_to_absorption='all')
g.plot_absorption_probabilities(same_plot=False, basis="umap", perc=[0, 99], ncols=2)

In [ ]:
scv.set_figure_params('scvelo', transparent=True, fontsize=20, color_map='viridis')

cr.pl.aggregate_absorption_probabilities(
    adata,
    mode='heatmap',
    lineages=["Acinar", "Ductal", "Alpha", "Beta", "Delta", "Epsilon"],
    cluster_key='celltype',
    clusters=['Acinar', 'Multipotent', 'Tip', 'Ductal', 'Ngn3 High early','Ngn3 High late', 'Fev+ Alpha', 'Alpha', 'Fev+ Beta', 'Beta', 'Fev+ Delta', 'Delta', 'Fev+ Epsilon', 'Epsilon'],
    figsize=(17, 6),
    title="",
    #save="transprobs_pancreas1415_pmm+gr.png"
)

In [ ]:
adata.uns["velocity_graph"] = vk.transition_matrix
scv.pl.velocity_embedding_stream(adata, basis="umap", title="", smooth=0.5, legend_loc='none')

In [ ]:
adata.layers["Ms"] = adata.layers["pca"].copy()
scv.tl.velocity_confidence(adata)

In [ ]:
scv.set_figure_params('scvelo', fontsize=20)
scv.settings.presenter_view = False

In [ ]:
scv.pl.scatter(adata, color='velocity_confidence',
               perc=[2, 98],
               cmap='gnuplot',
               vmid=0.75,
               rescale_color=[0.5, 1.0],
               size=15,
               color_map="none",
               title="",
               #save="velconf_pancreas1415_pmm+gr.png"
              )